OK. We've spent some time thinking about how to get and work with data, but we haven't really touched on what you can do with data once you have it. The reason for this is that data munging and data analysis are really two separate concepts in their own way. And the kinds of analysis you can perform on data are as vast as the types of data you could find. As a digital humanist, you might be interested in any number of things: georeferencing, statistical measurements, network analysis, or many more. And, then, once you've analyzed things, you'll likely want to visualize your results. For the purposes of showing you what you can do with Python, we will just barely scratch the surface of these areas by showing some very basic methods. We will then visualize our results, which will hopefully show how you can use programming to carry out interpretations. Our goal here will be to use some of the data from the previous lesson on web scraping. Since the previous data was text, we will be working with basic text analysis to analyze author style by word counts.

First let's get the data we need by copying over some of the work that we did last time. First we will import the modules that we need. Then we will use Beautiful Soup to scrape down the corpus. Be sure to check out the previous lesson or ask a neighbor if you have any questions about what any of these lines are doing. This will take a few moments.

In [ ]:
from bs4 import BeautifulSoup
from contextlib import closing
from urllib import request

url = "https://raw.githubusercontent.com/humanitiesprogramming/scraping-corpus/master/full-text.txt"
html = request.urlopen(url).read()
soup = BeautifulSoup(html, 'lxml')
raw_text = soup.text
texts = eval(soup.text)

The eval() function is new here, and it tells Python to take a string it is passed and interpret it as code. Beautiful Soup pulls down the contents of a website, but it assumes that the result of soup.text is going to be a string. If you actually look at the contents of that link, though, you'll see that I dumped the contents of the texts as a list of texts. So we need to interpret that big long text file as code, and Python can help us do that. Calling eval on it looks for special characters like the [], which indicate lists, and runs Python on it as expected. To actually work with this code again. We can prove that this is going on by taking the length of our two different versions of the soup results:

In [ ]:
print(len(raw_text))
print(len(texts))

The first len() function is way larger, as it is taking the length of a giant string. So it returns the total number of characters in the collected text. The second statement gives us the expected result "10", because it is measuring the length of our list of texts. We have 10 of them. As always, it is important that we remember what data types we have and when we have them.

Now that we have our data, we can start processing it as text. The package we are using is NLTK, the Natural Language Toolkit, which is something of a Swiss army knife for text analysis. Other packages might give you better baked in functionality, but NLTK is great for learning because it expects that you'll be working your own text functions from scratch. It also has a fantastic [text book](https://nltk.org/book) that I often use for teaching text analysis. The exercises rapidly engage you in real-world text questions. Let's start by importing what we'll need:

In [9]:
import nltk
from nltk import word_tokenize

NLTK is a massive package, with lots of moving pieces. We'll be calling lots of lower-level functions, so expect to do a lot of dot typing. The second line here is a way of shortening that process so that instead of typing nltk.word_tokenize a lot we can just type word_tokenize. Before we work with NLTK, we'll actually need to download some more things to your computer. The following line will call a pop-up downloader interface so that we can download a bunch of texts baked into NLTK. You probably only need to run it once.

In [6]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
removing collection member with no package: panlex_lite


True

Now that we've pulled texts onto your computer, NLTK will give us access to them if we bring them into Python. Here we pull in the first text and the first ten words.

In [12]:
from nltk.book import *
print(text1)
text1[0:10]

<Text: Moby Dick by Herman Melville 1851>


['[',
 'Moby',
 'Dick',
 'by',
 'Herman',
 'Melville',
 '1851',
 ']',
 'ETYMOLOGY',
 '.']

How fun! But wait, something is going on here - this text looks different than the raw text that we have. Sentences vs words vs bulk text. Segmentation and tokenization

TODO: Introduce basics of the NLP pipeline - normalizing, tokenizing, stopwords
TODO: Get some basic results
TODO: Visualize them with matplotlib